In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [3]:
train_df = pd.read_csv('../input/digit-recognizer/train.csv')

In [16]:
train_df = train_df[train_df['label']==8]

In [17]:
X = train_df.drop(columns='label')

In [18]:
X_train,X_test = train_test_split(X,test_size=0.2,random_state=1)

In [19]:
X_train,X_test = X_train.to_numpy(),X_test.to_numpy()

In [20]:
plt.imshow(X_train[0].reshape(28,28))

In [21]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Model, load_model # for creating a Neural Network Autoencoder model
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, LeakyReLU, BatchNormalization # for adding layers to AE model
from tensorflow.keras.utils import plot_model # for plotting model diagram

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # For rescaling metrics to fit into 0 to 1 range
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples

In [22]:
#Define shape
N_input = X_train.shape[1]
N_bottleneck = X_train.shape[1]/2

#Define layers
#Input layer
visible = Input(shape=(N_input,), name='Input-Layer')

#Encoder layer
e = Dense(units=N_input,name='Encoder-Layer')(visible)
e = BatchNormalization(name='Encoder-Layer-Normalization')(e)
e = LeakyReLU(name='Encoder-Layer-Activation')(e)

#--- Bottleneck
bottleneck = Dense(units=N_bottleneck, name='Bottleneck-Layer')(e)

#--- Decoder Layer
d = Dense(units=N_input, name='Decoder-Layer')(bottleneck)
d = BatchNormalization(name='Decoder-Layer-Normalization')(d)
d = LeakyReLU(name='Decoder-Layer-Activation')(d)

#--- Output layer
output = Dense(units=N_input, activation='linear', name='Output-Layer')(d)


# Define autoencoder model
model = Model(inputs=visible, outputs=output, name='Autoencoder-Model')

# Compile autoencoder model
model.compile(optimizer='adam', loss='mse')

# Print model summary
print(model.summary())

# Plot the autoencoder model diagram
plot_model(model, to_file='./Autoencoder.png', show_shapes=True, dpi=300)

In [24]:
# Fit the autoencoder model to reconstruct input
history = model.fit(X_train, X_train, epochs=40, batch_size=16, verbose=1, validation_data=(X_test, X_test))

# Plot a loss chart
fig, ax = plt.subplots(figsize=(16,9), dpi=300)
plt.title(label='Model Loss by Epoch', loc='center')

ax.plot(history.history['loss'], label='Training Data', color='black')
ax.plot(history.history['val_loss'], label='Test Data', color='red')
ax.set(xlabel='Epoch', ylabel='Loss')
plt.legend()

plt.show()

In [25]:
# Define an encoder model without the decoder
encoder = Model(inputs=visible, outputs=output)

# Compile encoder model
encoder.compile(optimizer='adam', loss='mse')

# Save the encoder model to file
encoder.save('./decoderv1.h5')

# Plot the autoencoder model diagram
plot_model(encoder, to_file='./decoderv1.png', show_shapes=True, dpi=300)

In [26]:
# Load the model from file
decoder = load_model('../input/decoder-on-8/decoderv1.h5')

In [27]:
X_train.shape

In [29]:
noise = np.ones((10,784))
decoded_noise = decoder.predict(noise)

In [30]:
plt.imshow(decoded_noise[0].reshape(28,28))

In [32]:
noise = np.random.rand(10,784)
decoded_noise = decoder.predict(noise)

In [38]:
plt.imshow(decoded_noise[0].reshape(28,28))